# logging package
Practice the usage of Python logging.

## References
* Python Logging: How to Write Logs Like a Pro!
    * https://youtu.be/pxuXaaT1u3k
* Python Logging - Tutorial
    * https://youtu.be/urrfJgHwIJA
* By Corey Schafer
    * Python Tutorial: Logging Basics - Logging to Files, Setting Levels, and Formatting
        * https://youtu.be/-ARI4Cz-awo
    * Python Tutorial: Logging Advanced - Loggers, Handlers, and Formatters
        * https://youtu.be/jxmzY9soFXg

<hr style="height:2px;background-color:black">

## Simple usages

In [1]:
import logging

# Configure the logging
# Once the basicConfig is set, it is effective for the whole session.
# To apply a new configuration, restart the kernel.
logging_level = logging.WARNING
# For attribute names that goes into the format string,
#   see https://docs.python.org/3/library/logging.html#logrecord-attributes
formatstr = "%(name)s: %(asctime)s %(levelname)-8s: %(message)s"
datefmt="%Y-%m-%d %H:%M:%S"
logging.basicConfig(level=logging_level, format=formatstr, datefmt=datefmt)

# Instantiate a Logger object
logger = logging.getLogger('mylogger')

# Test my logger
# Currently level is set to WARNING
logger.debug('Debug')
logger.warning('Warning')
print('Change logging level. This print message does not come inbetween log messages in stdout')
# Set logging level to DEBUG
logger.setLevel(logging.DEBUG)
logger.debug('Debug')
logger.warning('Warning')

mylogger: 2023-02-17 12:48:37 WARNING : Warning
mylogger: 2023-02-17 12:48:37 DEBUG   : Debug
mylogger: 2023-02-17 12:48:37 WARNING : Warning


Change logging level. This print message does not come inbetween log messages in stdout


<hr style="height:2px;background-color:black">

# MPI tasks
## Format string to have rank number and all output to a single file

In [1]:
import logging
# Get mpi rank and size info if MPI is enabled
try:
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    mpi_rank = comm.Get_rank()
    mpi_size = comm.Get_size()
except (ModuleNotFoundError, ImportError):
    print('MPI is not enabled. Use default single rank.')
    mpi_size = 1
    mpi_rank = 0

# Set up logger
logging_level = logging.WARNING
formatstr = f"Rank {mpi_rank:04d} of size {mpi_size:04d} [%(asctime)s] %(levelname)-8s: %(message)s"
datefmt = "%Y-%m-%d %H:%M:%S"
logging.basicConfig(level=logging_level, format=formatstr, datefmt=datefmt)

# Instantiate logger
logger = logging.getLogger('mylogger')

# Set up a handler that will write the log messages to a file as well as to stdout
filename = 'mpi_single_logger/mpi_single_logger.log'
filehandler = logging.FileHandler(filename)
filehandler.setLevel(logger.getEffectiveLevel())

# Lets make the output messages to file formatted the same as to console
formatter = logging.Formatter(formatstr, datefmt=datefmt)
filehandler.setFormatter(formatter)
logger.addHandler(filehandler)

# Log messages
logger.debug('Debug message')
logger.warning('Warning message')
print('Change logging level. This print message does not come inbetween log messages in stdout.')
# Set logging level to DEBUG
# (Note: but filehandler did not update its level, so the log in file does not contain the debug message)
logger.setLevel(logging.DEBUG)
logger.debug('Debug')
logger.warning('Warning')

Rank 0000 of size 0001 [2023-02-17 12:08:50] WARNING : Warning message
Rank 0000 of size 0001 [2023-02-17 12:08:50] DEBUG   : Debug
Rank 0000 of size 0001 [2023-02-17 12:08:50] WARNING : Warning


Change logging level. This print message does not come inbetween log messages in stdout


In [4]:
# The above test cell of the notebook, only one mpi task is used.
# For mpi run, we need to use a command like mpirun (see the next cell)

In [4]:
# Save above cell to mpi_single_logger/mpi_single_logger.py
# Then run the following
#   mpirun -n 4 python mpi_single_logger/mpi_single_logger.py
results = """
Rank 0000 of size 0004 [2023-02-17 12:15:34] WARNING : Warning message
Rank 0001 of size 0004 [2023-02-17 12:15:34] WARNING : Warning message
Rank 0001 of size 0004 [2023-02-17 12:15:34] DEBUG   : Debug
Rank 0002 of size 0004 [2023-02-17 12:15:34] WARNING : Warning message
Rank 0002 of size 0004 [2023-02-17 12:15:34] DEBUG   : Debug
Rank 0003 of size 0004 [2023-02-17 12:15:34] WARNING : Warning message
Rank 0003 of size 0004 [2023-02-17 12:15:34] DEBUG   : Debug
Rank 0000 of size 0004 [2023-02-17 12:15:34] DEBUG   : Debug
Rank 0003 of size 0004 [2023-02-17 12:15:34] WARNING : Warning
Rank 0001 of size 0004 [2023-02-17 12:15:34] WARNING : Warning
Change logging level. This print message does not come inbetween log messages in stdout.
Change logging level. This print message does not come inbetween log messages in stdout.
Rank 0000 of size 0004 [2023-02-17 12:15:34] WARNING : Warning
Rank 0002 of size 0004 [2023-02-17 12:15:34] WARNING : Warning
Change logging level. This print message does not come inbetween log messages in stdout.
Change logging level. This print message does not come inbetween log messages in stdout.
"""

# The saved file doesn't necessarily show the results in the same order
# cat mpi_single_logger/mpi_single_logger.log
results = """
Rank 0002 of size 0004 [2023-02-17 12:15:34] WARNING : Warning message
Rank 0000 of size 0004 [2023-02-17 12:15:34] WARNING : Warning message
Rank 0001 of size 0004 [2023-02-17 12:15:34] WARNING : Warning message
Rank 0003 of size 0004 [2023-02-17 12:15:34] WARNING : Warning message
Rank 0003 of size 0004 [2023-02-17 12:15:34] WARNING : Warning
Rank 0001 of size 0004 [2023-02-17 12:15:34] WARNING : Warning
Rank 0000 of size 0004 [2023-02-17 12:15:34] WARNING : Warning
Rank 0002 of size 0004 [2023-02-17 12:15:34] WARNING : Warning
"""

# I can sort out each rank's log by grepping as well
# grep 'Rank 0001' mpi_single_logger/mpi_single_logger.log
results = """
Rank 0001 of size 0004 [2023-02-17 12:15:34] WARNING : Warning message
Rank 0001 of size 0004 [2023-02-17 12:15:34] WARNING : Warning
"""

## Each rank to have a seprate logger and an output file
For embarassingly parallel jobs, it may be useful and enough to make the log for each mpi task separately.

<font color="blue">Or, is this necessary? Can I just grep as above?</font> Slurm (see below) outputs to .err and .out as a single file anyway (as far as I know and have tried).

In [1]:
import logging
# Get mpi rank and size info if MPI is enabled
try:
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    mpi_rank = comm.Get_rank()
    mpi_size = comm.Get_size()
except (ModuleNotFoundError, ImportError):
    print('MPI is not enabled. Use default single rank.')
    mpi_size = 1
    mpi_rank = 0

# Set up logger
logging_level = logging.WARNING
formatstr = f"Rank {mpi_rank:04d} of size {mpi_size:04d} [%(asctime)s] %(levelname)-8s: %(message)s"
datefmt = "%Y-%m-%d %H:%M:%S"
logging.basicConfig(level=logging_level, format=formatstr, datefmt=datefmt)

# Instantiate logger
logger = logging.getLogger(f'rank{mpi_rank:04d}logger')

# Set up a handler that will write the log messages to a file as well as to stdout
filename = f'mpi_logger_per_rank/mpi_rank{mpi_rank:04d}.log'
filehandler = logging.FileHandler(filename)
filehandler.setLevel(logger.getEffectiveLevel())

# Lets make the output messages to file formatted the same as to console
formatter = logging.Formatter(formatstr, datefmt=datefmt)
filehandler.setFormatter(formatter)
logger.addHandler(filehandler)

# Log messages
logger.debug('Debug message')
logger.warning('Warning message')
print('Change logging level. This print message does not come inbetween log messages in stdout.')
# Set logging level to DEBUG
# (Note: but filehandler did not update its level, so the log in file does not contain the debug message)
logger.setLevel(logging.DEBUG)
logger.debug('Debug')
logger.warning('Warning')

Rank 0000 of size 0001 [2023-02-17 12:18:34] WARNING : Warning message
Rank 0000 of size 0001 [2023-02-17 12:18:34] DEBUG   : Debug
Rank 0000 of size 0001 [2023-02-17 12:18:34] WARNING : Warning


Change logging level. This print message does not come inbetween log messages in stdout.


In [5]:
# The above test cell of the notebook, only one mpi task is used.
# For mpi run, we need to use a command like mpirun (see the next cell)

In [2]:
# Save above cell to mpi_logger_per_rank/mpi_logger_per_rank.py
# Then run the following
#   mpirun -n 4 python mpi_logger_per_rank/mpi_logger_per_rank.py
results = """
Rank 0000 of size 0004 [2023-02-17 12:19:04] WARNING : Warning message
Rank 0000 of size 0004 [2023-02-17 12:19:04] DEBUG   : Debug
Rank 0002 of size 0004 [2023-02-17 12:19:04] WARNING : Warning message
Rank 0002 of size 0004 [2023-02-17 12:19:04] DEBUG   : Debug
Rank 0000 of size 0004 [2023-02-17 12:19:04] WARNING : Warning
Rank 0002 of size 0004 [2023-02-17 12:19:04] WARNING : Warning
Change logging level. This print message does not come inbetween log messages in stdout.
Change logging level. This print message does not come inbetween log messages in stdout.
Rank 0001 of size 0004 [2023-02-17 12:19:04] WARNING : Warning message
Rank 0003 of size 0004 [2023-02-17 12:19:04] WARNING : Warning message
Rank 0001 of size 0004 [2023-02-17 12:19:04] DEBUG   : Debug
Rank 0003 of size 0004 [2023-02-17 12:19:04] DEBUG   : Debug
Change logging level. This print message does not come inbetween log messages in stdout.
Rank 0001 of size 0004 [2023-02-17 12:19:04] WARNING : Warning
Rank 0003 of size 0004 [2023-02-17 12:19:04] WARNING : Warning
Change logging level. This print message does not come inbetween log messages in stdout.
"""
# The saved file contains only the logs that belong to the rank.
# mpi_logger_per_rank/mpi_rank0000.log
# mpi_logger_per_rank/mpi_rank0001.log
# mpi_logger_per_rank/mpi_rank0002.log
# mpi_logger_per_rank/mpi_rank0003.log
# e.g. cat mpi_logger_per_rank/mpi_rank0000.log
results = """
Rank 0000 of size 0004 [2023-02-17 12:19:04] WARNING : Warning message
Rank 0000 of size 0004 [2023-02-17 12:19:04] WARNING : Warning
"""


<hr style="height:2px;background-color:black">

# Slurm stderr and stdout
Control how to send log messages for a slurm job.

By default, the log messages go to .err file (stderr)
* https://docs.python.org/3/library/logging.handlers.html#streamhandler

<hr style="height:2px;background-color:black">